In [1]:
%config IPCompleter.greedy=True

In [2]:
import numpy as np
import re
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
import sklearn.utils

In [3]:
cachedStopWords = stopwords.words("english")

In [4]:
#df = pd.DataFrame.from_csv('mbti_1.csv')
df = pd.DataFrame.from_csv('modified_data.csv')

In [22]:
'''Balancing dataset'''
total_examples = len(df)
categories = df['type'].unique()
num_classes = len(categories)
examples_per_type = np.floor(total_examples / num_classes).astype(int)

frames = []
for category in categories:
    tmp_df = df[df['type'] == category]
    tmp_df = tmp_df.head(examples_per_type)
    frames.append(tmp_df)
    #print(len(tmp_df))
df = pd.concat(frames).reset_index(drop=True)

In [293]:
#groups = df.groupby('type')
#for g, gdf in groups:
    #print(gdf['type'].count())

In [35]:
df = sklearn.utils.shuffle(df).reset_index(drop=True)
df = sklearn.utils.shuffle(df).reset_index(drop=True)
df = sklearn.utils.shuffle(df).reset_index(drop=True)

In [300]:
#for i in range(5):
#    df = df.sample(frac=1).reset_index(drop=True)

In [46]:
length = len(df)
train_data_len = np.floor(length * 0.6).astype(int)
#train_data = df.sample(frac=0.9).reset_index(drop=True)
#test_data = df.sample(frac=0.1).reset_index(drop=True)
train_data = df.tail(train_data_len)
test_data = df.head(length - train_data_len)

In [37]:
train_data['type'].nunique()

16

In [38]:
test_data['type'].nunique()

16

In [47]:
'''Initializing a couter for each of the MBTI category'''
mbti_types = train_data['type'].unique()
mbti_counter = dict()

for mbti_type in mbti_types:
    mbti_counter[mbti_type] = Counter()

In [159]:
#train_data.to_csv('train_data.csv', sep='\t', encoding='utf-8')
#test_data.to_csv('test_data.csv', sep='\t', encoding='utf-8')

In [48]:
n = len(train_data)
for i in range(n):
    #if i > 5:
    #    break
    #text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', train_data.iloc[i]['posts'])
    tokens = re.findall(r"[\w']+", train_data.iloc[i]['posts'])
    #print(tokens)
    #tokens = [word for word in tokens if word not in cachedStopWords and len(word) >= 3]
    #print(tokens)
    mbti_counter[train_data.iloc[i]['type']].update(tokens)
    #print(mbti_counter[data.iloc[i]['type']])
    #print()

In [49]:
'''Calculate the class probability'''
class_prob = {}
total = len(train_data)
for mbti_type in mbti_types:
    class_prob[mbti_type] = np.log(train_data[train_data['type'] == mbti_type]['type'].count() / total)

In [50]:
'''Calculate the feature probability'''
feature_prob = {}
alpha = 1.0
num_classes = len(train_data['type'].unique())
for mbti_type in mbti_types:
    total = sum(mbti_counter[mbti_type].values())
    prob = {}
    for token in mbti_counter[mbti_type]:
        prob[token] = np.log((alpha + mbti_counter[mbti_type][token]) / (total + num_classes * alpha))
    feature_prob[mbti_type] = prob

In [51]:
model = {}
model['class_prob'] = class_prob
model['feature_prob'] = feature_prob

In [52]:
'''Prediction accuracy on traning data'''
def naive_bayes_predict(example, model):
    #print(example)
    #text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', example)
    tokens = re.findall(r"[\w']+", example)
    
    min_prob = 0
    min_label = ''
    
    for mbti_type in mbti_types:
        prob = 1.0
        for token in tokens:
            token_prob = 1.0
            try:
                token_prob = model['feature_prob'][mbti_type][token]
            except KeyError:
                token_prob = 1.0

            prob = prob + token_prob
        prob += model['class_prob'][mbti_type]

        if prob < min_prob:
            min_prob = prob
            min_label = mbti_type
    
    return min_label

#test_data = pd.DataFrame.from_csv('mbti_1.csv')
#test_data = test_data.sample(frac=0.1).reset_index(drop=True)
labels = []
n = len(test_data)
for i in range(n):
    label = naive_bayes_predict(test_data.iloc[i]['posts'], model)
    labels.append(label)

In [53]:
actual_labels = test_data['type'].tolist()
total = 0
for i in range(len(labels)):
    if actual_labels[i] == labels[i]:
        total += 1
print(total / len(labels))

0.16491537895511404
